<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/Timescalevector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Timescale Vector 存储（PostgreSQL）

本笔记本展示了如何使用Postgres向量存储`TimescaleVector`来存储和查询向量嵌入。

## 什么是Timescale Vector？
**[Timescale Vector](https://www.timescale.com/ai) 是面向AI应用的PostgreSQL++。**

Timescale Vector使您能够在`PostgreSQL`中高效地存储和查询数百万个向量嵌入。
- 通过受DiskANN启发的索引算法，增强了`pgvector`，实现了更快速和更准确的相似性搜索。
- 通过自动基于时间的分区和索引，实现了快速的基于时间的向量搜索。
- 提供了一个熟悉的SQL接口，用于查询向量嵌入和关系数据。

Timescale Vector可以随着您的需求从POC扩展到生产环境：
- 通过使您能够在单个数据库中存储关系元数据、向量嵌入和时间序列数据，简化了操作。
- 借助企业级功能（如流式备份和复制、高可用性和行级安全性），从坚固的PostgreSQL基础中受益。
- 通过企业级安全性和合规性，实现了无忧体验。

## 如何使用Timescale Vector
Timescale Vector可在[Timescale](https://www.timescale.com/ai)上使用，这是一个云端的PostgreSQL平台。（目前没有自托管版本。）

**LlamaIndex用户可以获得Timescale Vector的90天免费试用。**
- 要开始使用，请[注册](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch&utm_source=llamaindex&utm_medium=referral)Timescale，创建一个新数据库，然后按照本笔记本的步骤进行操作！
- 有关详细信息和性能基准，请参阅[Timescale Vector解释博客](https://www.timescale.com/blog/how-we-made-postgresql-the-best-vector-database/?utm_campaign=vectorlaunch&utm_source=llamaindex&utm_medium=referral)。
- 有关在python中使用Timescale Vector的更多详细信息，请参阅[安装说明](https://github.com/timescale/python-vector)。


## 0. 设置
让我们导入这个笔记本所需的所有内容。


如果您在Colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-embeddings-openai
%pip install llama-index-vector-stores-timescalevector

In [ ]:
!pip install llama-index

In [ ]:
# 导入日志模块
# 导入系统模块

# 取消注释以查看调试日志
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from  timescale_vector
from  llama_index.core  import  SimpleDirectoryReader, StorageContext
from  llama_index.core  import  VectorStoreIndex
from  llama_index.vector_stores.timescalevector  import  TimescaleVectorStore
from  llama_index.core.vector_stores  import  VectorStoreQuery, MetadataFilters
from  textwrap
from  openai

### 设置OpenAI API密钥
为了为加载到索引中的文档创建嵌入，让我们配置您的OpenAI API密钥：


In [ ]:
# 通过读取本地的 .env 文件获取 openAI 的 API 密钥
# .env 文件应包含以 `OPENAI_API_KEY=sk-` 开头的一行
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

# 或者显式设置
# import os
# os.environ["OPENAI_API_KEY"] = "<your key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

### 创建一个PostgreSQL数据库并获取Timescale服务URL
您需要一个服务URL来连接到您的Timescale数据库实例。

首先，在[Timescale](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch&utm_source=llamaindex&utm_medium=referral)中启动一个新的云数据库（使用上面的链接免费注册）。

要连接到您的云PostgreSQL数据库，您需要您的服务URI，这可以在创建新数据库后下载的速查表或`.env`文件中找到。

URI看起来会像这样：`postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require`


In [ ]:
# 通过读取本地的.env文件来获取服务的URL
# .env文件应该包含一行以`TIMESCALE_SERVICE_URL=postgresql://`开头的内容
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

TIMESCALE_SERVICE_URL = os.environ["TIMESCALE_SERVICE_URL"]

# 或者可以显式设置
# TIMESCALE_SERVICE_URL = "postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require"

## 1. 使用时间刻度向量进行简单相似度搜索


```python
# 下载数据
```

在这个部分，我们将学习如何下载数据。


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 加载文档
在本例中，我们将使用[SimpleDirectoryReader](https://gpt-index.readthedocs.io/en/stable/examples/data_connectors/simple_directory_reader.html)来加载存储在`paul_graham_essay`目录中的文档。

`SimpleDirectoryReader`是LlamaIndex中最常用的数据连接器之一，用于从目录中读取一个或多个文件。


In [ ]:
# 从数据目录使用SimpleDirectoryReader加载样本数据
documents = SimpleDirectoryReader("./data/paul_graham").load_data()
print("文档ID:", documents[0].doc_id)

Document ID: 740ce1a1-4d95-40cc-b7f7-6d2874620a53


### 使用TimescaleVectorStore创建VectorStore索引
接下来，为了执行相似性搜索，我们首先创建一个`TimescaleVector` [vector store](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html)，用于存储我们从文章内容中提取的向量嵌入。TimescaleVectorStore需要一些参数，包括我们上面加载的`service_url`，以及一个`table_name`，这将是存储向量的表的名称。

然后，我们在由先前的文档支持的Timescale上创建一个[Vector Store Index](https://gpt-index.readthedocs.io/en/stable/community/integrations/vector_stores.html#vector-store-index)。


In [ ]:
# 创建一个TimescaleVectorStore来存储文档
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

# 使用TimescaleVectorStore创建一个新的VectorStoreIndex
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### 查询索引
现在我们已经在VectorStore中索引了文档，我们可以使用默认的`query_engine`来查询索引中的文档。

请注意，您还可以配置查询引擎以配置返回的前k个最相似的结果，以及通过元数据过滤器来过滤结果。有关更多详细信息，请参阅[配置标准查询设置部分](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/index/vector_store_guide.html)。


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Did the author work at YC?")

In [ ]:
print(textwrap.fill(str(response), 100))

Yes, the author did work at YC.


In [ ]:
response = query_engine.query("What did the author work on before college?")

In [ ]:
print(textwrap.fill(str(response), 100))

Before college, the author worked on writing and programming. They wrote short stories and also
tried programming on the IBM 1401 computer using an early version of Fortran.


### 查询现有索引
在上面的示例中，我们从加载的文档中创建了一个新的Timescale Vector vectorstore和索引。接下来，我们将看看如何查询现有的索引。我们只需要服务URI和要访问的表名。


In [ ]:
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do before YC?")

In [ ]:
print(textwrap.fill(str(response), 100))

Before YC, the author wrote all of YC's internal software in Arc. They also worked on HN and had
three projects: writing essays, working on YC, and working in Arc. However, they gradually stopped
working on Arc due to time constraints and the increasing dependence on it for infrastructure.


## 2. 使用ANN搜索索引加速查询

（注意：这些索引是ANN索引，与LlamaIndex中的索引概念不同）

您可以通过在嵌入列上创建索引来加速相似性查询。只有在摄入了大部分数据后才应该这样做。

Timescale Vector支持以下索引：
- timescale_vector_index：用于快速相似性搜索的磁盘ANN启发式图索引（默认）。
- pgvector的HNSW索引：用于快速相似性搜索的分层可导航小世界图索引。
- pgvector的IVFFLAT索引：用于快速相似性搜索的倒排文件索引。

重要提示：在PostgreSQL中，每个表只能在特定列上有一个索引。因此，如果您想测试不同索引类型的性能，可以通过（1）创建具有不同索引的多个表，（2）在同一表中创建多个向量列并在每个列上创建不同的索引，或者（3）删除并重新创建相同列上的索引并比较结果来进行测试。


In [ ]:
# 从第1部分实例化TimescaleVectorStore
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

使用`create_index()`函数而不带额外参数将默认创建一个`timescale_vector (DiskANN)`索引，使用默认参数。


In [ ]:
# 创建一个时间刻度向量索引（DiskANN）
vector_store.create_index()

您还可以为索引指定参数。请参阅Timescale Vector文档，了解有关不同参数及其对性能的影响的全面讨论。


In [ ]:
# 删除旧索引
vector_store.drop_index()

# 使用指定参数创建新的时间刻度向量索引（DiskANN）
vector_store.create_index("tsv", max_alpha=1.0, num_neighbors=50)

Timescale Vector还支持HNSW和ivfflat索引：


In [ ]:
vector_store.drop_index()

# 创建一个HNSW索引
# 注意：您不需要指定m和ef_construction参数，因为我们设置了智能默认值。
vector_store.create_index("hnsw", m=16, ef_construction=64)

In [ ]:
# 创建一个IVFFLAT索引
# 注意：您不需要指定num_lists和num_records参数，因为我们设置了智能默认值。
vector_store.drop_index()
vector_store.create_index("ivfflat", num_lists=20, num_records=1000)

我们一般建议在使用时选择`timescale-vector`或`HNSW`索引。


In [ ]:
# 删除ivfflat索引
vector_store.drop_index()
# 创建一个timescale向量索引（DiskANN）
vector_store.create_index()

## 3. 基于时间过滤的相似性搜索

Timescale Vector的一个关键用例是高效的基于时间的向量搜索。Timescale Vector通过自动按时间对向量（和相关元数据）进行分区来实现这一点。这使您能够通过相似度和时间高效地查询向量。

基于时间的向量搜索功能对于以下应用程序很有帮助：
- 存储和检索LLM响应历史（例如聊天机器人）
- 查找与查询向量相似的最新嵌入（例如最近的新闻）
- 将相似性搜索限制在相关的时间范围内（例如在知识库中询问基于时间的问题）

为了说明如何使用TimescaleVector的基于时间的向量搜索功能，我们将使用TimescaleDB的git日志历史作为样本数据集，并对其进行提问。每个git提交条目都有与之关联的时间戳，以及自然语言消息和其他元数据（例如作者、提交哈希等）。

我们将说明如何创建具有基于时间的uuid的节点，并如何使用TimescaleVector vectorstore运行带有时间范围过滤器的相似性搜索。


### 从git log CSV文件中提取内容和元数据

首先，让我们将git log csv文件加载到我们的PostgreSQL数据库中的一个名为`timescale_commits`的新集合中。

注意：由于这只是一个演示，我们将只处理前1000条记录。在实际应用中，您可以加载任意数量的记录。


In [ ]:

import pandas as pd
from pathlib import Path

file_path = Path("../data/csv/commit_history.csv")
# 读取CSV文件到DataFrame中
df = pd.read_csv(file_path)

# 对CSV进行轻微的数据清洗
df.dropna(inplace=True)
df = df.astype(str)
df = df[:1000]

In [ ]:
# 查看csv中的数据（可选）
df.head()

我们将定义一个辅助函数，根据节点的时间戳创建一个uuid，并关联一个向量嵌入。我们将使用这个函数为每个git日志条目创建一个uuid。

重要说明：如果您正在处理文档/节点，并希望将当前日期和时间与向量关联起来进行基于时间的搜索，您可以跳过这一步。在Timescale Vector中，节点被默认添加到表中时会自动生成一个uuid。在我们的情况下，因为我们希望uuid是基于过去时间戳的，所以我们需要手动创建uuid。


In [ ]:
from timescale_vector import client

# 接受过去日期字符串并返回uuid v1的函数
def create_uuid(date_string: str):
    if date_string is None:
        return None
    time_format = "%a %b %d %H:%M:%S %Y %z"
    datetime_obj = datetime.strptime(date_string, time_format)
    uuid = client.uuid_from_time(datetime_obj)
    return str(uuid)

In [ ]:
# 辅助函数
from typing import List, Tuple


# 辅助函数，用于根据包含姓名、姓和邮箱的作者字符串拆分姓名和邮箱
def split_name(input_string: str) -> Tuple[str, str]:
    if input_string is None:
        return None, None
    start = input_string.find("<")
    end = input_string.find(">")
    name = input_string[:start].strip()
    return name


from datetime import datetime, timedelta


def create_date(input_string: str) -> datetime:
    if input_string is None:
        return None
    # 定义一个字典，将月份的缩写映射到它们的数字等价物
    month_dict = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "May": "05",
        "Jun": "06",
        "Jul": "07",
        "Aug": "08",
        "Sep": "09",
        "Oct": "10",
        "Nov": "11",
        "Dec": "12",
    }

    # 将输入字符串拆分为其组成部分
    components = input_string.split()
    # 提取相关信息
    day = components[2]
    month = month_dict[components[1]]
    year = components[4]
    time = components[3]
    timezone_offset_minutes = int(
        components[5]
    )  # 将偏移量转换为分钟
    timezone_hours = timezone_offset_minutes // 60  # 计算小时
    timezone_minutes = (
        timezone_offset_minutes % 60
    )  # 计算剩余的分钟
    # 创建一个格式化字符串，用于PostgreSQL格式的timestamptz
    timestamp_tz_str = (
        f"{year}-{month}-{day} {time}+{timezone_hours:02}{timezone_minutes:02}"
    )
    return timestamp_tz_str

接下来，我们将定义一个函数，为每个git日志条目创建一个`TextNode`。我们将使用上面定义的辅助函数`create_uuid()`根据时间戳为每个节点创建一个uuid。我们还将使用上面的辅助函数`create_date()`和`split_name()`来从git日志条目中提取相关的元数据，并将它们添加到节点中。


In [ ]:
from llama_index.core.schema import TextNode,NodeRelationship和RelatedNodeInfo


# 从单行数据创建一个Node对象
def create_node(row):
    record = row.to_dict()
    record_name = split_name(record["author"])
    record_content = (
        str(record["date"])
        + " "
        + record_name
        + " "
        + str(record["change summary"])
        + " "
        + str(record["change details"])
    )
    # 根据需要可以更改为TextNode
    node = TextNode(
        id_=create_uuid(record["date"]),
        text=record_content,
        metadata={
            "commit": record["commit"],
            "author": record_name,
            "date": create_date(record["date"]),
        },
    )
    return node

In [ ]:
nodes = [create_node(row) for _, row in df.iterrows()]

接下来，我们将为每个节点的内容创建向量嵌入，以便我们可以对每个节点关联的文本进行相似性搜索。我们将使用`OpenAIEmbedding`模型来创建这些嵌入。


In [ ]:
# 为节点创建嵌入
from llama_index.embeddings.openai import OpenAIEmbedding

embedding_model = OpenAIEmbedding()

for node in nodes:
    node_embedding = embedding_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

让我们检查一下我们集合中的第一个节点，看看它是什么样子。


In [ ]:
print(nodes[0].get_content(metadata_mode="all"))

commit: 44e41c12ab25e36c202f58e068ced262eadc8d16
author: Lakshmi Narayanan Sreethar
date: 2023-09-5 21:03:21+0850

Tue Sep 5 21:03:21 2023 +0530 Lakshmi Narayanan Sreethar Fix segfault in set_integer_now_func When an invalid function oid is passed to set_integer_now_func, it finds out that the function oid is invalid but before throwing the error, it calls ReleaseSysCache on an invalid tuple causing a segfault. Fixed that by removing the invalid call to ReleaseSysCache.  Fixes #6037


In [ ]:
print(nodes[0].get_embedding())

[-0.005366453900933266, 0.0016374519327655435, 0.005981510039418936, -0.026256779208779335, -0.03944991156458855, 0.026299940422177315, -0.0200558640062809, -0.01252412423491478, -0.04241368919610977, -0.004758591763675213, 0.05639812350273132, 0.006578581873327494, 0.014833281747996807, 0.009509989991784096, 0.0009675443288870156, -0.013157163746654987, -0.002265996066853404, -0.017048921436071396, 0.006553404498845339, -0.00217068032361567, 0.009085564874112606, 0.011775985360145569, -0.02514895796775818, -0.002679630182683468, 0.0030608929228037596, -3.439458305365406e-05, -0.00363818253390491, -0.03939236328005791, 0.0016806137282401323, -0.01207092497497797, 0.01739421673119068, -0.02241537719964981, -0.01753808930516243, -0.023782167583703995, -0.01598426327109337, -0.02575322426855564, -0.016876274719834328, -0.006380756851285696, -0.0009149408433586359, 0.00704616867005825, -0.0013290246715769172, -0.009776154533028603, -0.013200325891375542, -0.024832438677549362, -0.001940483

### 将文档和元数据加载到TimescaleVector向量存储中
现在我们已经准备好了节点并为它们添加了嵌入，让我们将它们添加到我们的TimescaleVector向量存储中。

我们将从我们创建的节点列表中创建一个Timescale Vector实例。

首先，我们将定义一个集合名称，这将是我们在PostgreSQL数据库中的表名。

我们还将定义一个时间间隔，将其传递给`time_partition_interval`参数，该参数将用作按时间对数据进行分区的间隔。每个分区将包含指定时间长度的数据。我们将简单地使用7天，但您可以根据您的用例选择任何合理的值--例如，如果您经常查询最近的向量，则可能希望使用较小的时间间隔，如1天；或者如果您查询长达十年的向量，则可能希望使用较大的时间间隔，如6个月或1年。

然后我们将节点添加到Timescale Vector向量存储中。


In [ ]:
# 创建一个时间刻度向量存储，并将新创建的节点添加到其中
ts_vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="li_commit_history",
    time_partition_interval=timedelta(days=7),
)
_ = ts_vector_store.add(nodes)

### 按时间和相似度查询向量

现在我们已经将文档加载到TimescaleVector中，我们可以通过时间和相似度对它们进行查询。

TimescaleVector提供了多种方法来通过时间进行过滤并进行相似度搜索。让我们逐个查看每种方法。

首先，我们定义一个查询字符串并获取该查询字符串的向量嵌入。


In [ ]:
# 定义查询并为其生成嵌入
query_str = "TimescaleDB函数有什么新功能？"
embed_model = OpenAIEmbedding()
query_embedding = embed_model.get_query_embedding(query_str)

然后我们设置一些变量，这些变量将用于我们的时间过滤器中。


In [ ]:
# 查询的时间过滤变量
start_dt = datetime(
    2023, 8, 1, 22, 10, 35
)  # 开始日期 = 2023年8月1日，22:10:35
end_dt = datetime(
    2023, 8, 30, 22, 10, 35
)  # 结束日期 = 2023年8月30日，22:10:35
td = timedelta(days=7)  # 时间差 = 7天

方法1：在提供的开始日期和结束日期内进行筛选。


In [ ]:
# 查询向量数据库
vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

# 返回在开始日期和结束日期之间与查询最相似的向量
# 返回一个VectorStoreQueryResult对象
query_result = ts_vector_store.query(
    vector_store_query, start_date=start_dt, end_date=end_dt
)
query_result

VectorStoreQueryResult(nodes=[TextNode(id_='22747180-31f1-11ee-bd8e-101e36c28c91', embedding=None, metadata={'commit': ' 7aeed663b9c0f337b530fd6cad47704a51a9b2ec', 'author': 'Dmitry Simonenko', 'date': '2023-08-3 14:30:23+0500'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3273f20a98f02c75847896b929888b05e8751ae5e258d7feb8605bd5290ef8ca', text='Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), TextNode(id_='faa8ea00-4686-11ee-b933-c2c7df407c25', embedding=None, metadata={'commit': ' e4facda540286b0affba47ccc63959fefe2a7b26', 'author': 'Sven Klemm', 'date': '2023-08-29 18:13:2

让我们检查一下从相似性搜索中返回的节点：


In [ ]:
# 对查询结果中的每个节点，打印节点的元数据日期
for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))

--------------------------------------------------------------------------------
2023-08-3 14:30:23+0500
commit:  7aeed663b9c0f337b530fd6cad47704a51a9b2ec
author: Dmitry Simonenko
date: 2023-08-3 14:30:23+0500

Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create
--------------------------------------------------------------------------------
2023-08-29 18:13:24+0320
commit:  e4facda540286b0affba47ccc63959fefe2a7b26
author: Sven Klemm
date: 2023-08-29 18:13:24+0320

Tue Aug 29 18:13:24 2023 +0200 Sven Klemm Add compatibility layer for _timescaledb_internal functions With timescaledb 2.12 all the functions present in _timescaledb_internal were moved into the _timescaledb_functions schema to improve schema security. This patch adds a compatibility layer so external calle

注意查询只返回指定日期范围内的结果。


方法2：在提供的开始日期内进行筛选，并在一段时间间隔后进行筛选。


In [ ]:
vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

# 从开始日期和一段时间后返回与查询最相似的向量
query_result = ts_vector_store.query(
    vector_store_query, start_date=start_dt, time_delta=td
)

for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))


--------------------------------------------------------------------------------
2023-08-3 14:30:23+0500
commit:  7aeed663b9c0f337b530fd6cad47704a51a9b2ec
author: Dmitry Simonenko
date: 2023-08-3 14:30:23+0500

Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create
--------------------------------------------------------------------------------
2023-08-7 19:49:47+-500
commit:  5bba74a2ec083728f8e93e09d03d102568fd72b5
author: Fabrízio de Royes Mello
date: 2023-08-7 19:49:47+-500

Mon Aug 7 19:49:47 2023 -0300 Fabrízio de Royes Mello Relax strong table lock when refreshing a CAGG When refreshing a Continuous Aggregate we take a table lock on _timescaledb_catalog.continuous_aggs_invalidation_threshold when processing the invalidation logs (the first transaction of the refre

再次注意，只有在开始日期（8月1日）和定义的时间间隔之后（7天后）之间的节点才会被返回。


方法3：在提供的结束日期和比提前的时间间隔内进行筛选。


In [ ]:
# 创建一个VectorStoreQuery对象，使用query_embedding作为查询向量，返回相似度最高的5个结果
vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

# 从结束日期和时间间隔之前的时间查询与查询向量最相似的向量
query_result = ts_vector_store.query(
    vector_store_query, end_date=end_dt, time_delta=td
)

# 遍历查询结果中的节点
for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))


--------------------------------------------------------------------------------
2023-08-29 18:13:24+0320
commit:  e4facda540286b0affba47ccc63959fefe2a7b26
author: Sven Klemm
date: 2023-08-29 18:13:24+0320

Tue Aug 29 18:13:24 2023 +0200 Sven Klemm Add compatibility layer for _timescaledb_internal functions With timescaledb 2.12 all the functions present in _timescaledb_internal were moved into the _timescaledb_functions schema to improve schema security. This patch adds a compatibility layer so external callers of these internal functions will not break and allow for more flexibility when migrating.
--------------------------------------------------------------------------------
2023-08-29 10:49:47+0320
commit:  a9751ccd5eb030026d7b975d22753f5964972389
author: Sven Klemm
date: 2023-08-29 10:49:47+0320

Tue Aug 29 10:49:47 2023 +0200 Sven Klemm Move partitioning functions to _timescaledb_functions schema To increase schema security we do not want to mix our own internal objects with us

主要的要点是，在上面的每个结果中，只有在指定的时间范围内的向量才会被返回。这些查询非常高效，因为它们只需要搜索相关的分区。


## 4. 使用TimescaleVector存储作为检索器和查询引擎

现在我们已经探讨了基本的相似性搜索和带有基于时间的过滤器的相似性搜索，让我们看看如何将Timescale Vector的这些特性与LLamaIndex的检索器和查询引擎一起使用。

首先，我们将看看如何将TimescaleVector用作[检索器](https://gpt-index.readthedocs.io/en/latest/api_reference/query/retrievers.html)，具体来说是[矢量存储检索器](https://gpt-index.readthedocs.io/en/latest/api_reference/query/retrievers/vector_store.html)。

为了将检索到的节点限制在相关的时间范围内，我们可以使用TimescaleVector的时间过滤器。我们只需在创建检索器时将时间过滤器参数作为`vector_stored_kwargs`传递即可。


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

index = VectorStoreIndex.from_vector_store(ts_vector_store)
retriever = index.as_retriever(
    vector_store_kwargs=({"start_date": start_dt, "time_delta": td})
)
retriever.retrieve("What's new with TimescaleDB functions?")

[NodeWithScore(node=TextNode(id_='22747180-31f1-11ee-bd8e-101e36c28c91', embedding=None, metadata={'commit': ' 7aeed663b9c0f337b530fd6cad47704a51a9b2ec', 'author': 'Dmitry Simonenko', 'date': '2023-08-3 14:30:23+0500'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3273f20a98f02c75847896b929888b05e8751ae5e258d7feb8605bd5290ef8ca', text='Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.1813839050209377),
 NodeWithScore(node=TextNode(id_='b5583780-3574-11ee-871a-5a8c45d660c8', embedding=None, metadata={'commit': ' 5bba74a2ec083728f8e93e09d03d102568fd72b5', 'author': 'Fab

接下来，我们将看看如何将TimescaleVector用作[查询引擎](https://gpt-index.readthedocs.io/en/latest/api_reference/query/query_engines.html)。

同样地，我们使用TimescaleVector的时间过滤器来限制搜索范围，通过在创建查询引擎时将时间过滤器参数作为`vector_strored_kwargs`传递。


In [ ]:
index = VectorStoreIndex.from_vector_store(ts_vector_store)
query_engine = index.as_query_engine(
    vector_store_kwargs=({"start_date": start_dt, "end_date": end_dt})
)

# query_str = "What's new with TimescaleDB? List 3 new features"
query_str = (
    "TimescaleDB函数有什么新功能？这些变化是什么时候发生的，由谁进行的？"
)
response = query_engine.query(query_str)
print(str(response))

TimescaleDB functions have undergone changes recently. These changes were made by Sven Klemm on August 29, 2023. The changes involve adding a compatibility layer for _timescaledb_internal functions. This layer ensures that external callers of these internal functions will not break and allows for more flexibility when migrating.
